In [1]:
import os
import math
import pandas as pd
import numpy as np

data_dir = 'D:\\data\\caltecBirds\\CUB_200_2011'

In [2]:
# Load class and attribute data from the supplied text files

train_test_split = pd.read_csv(os.path.join(data_dir, 'train_test_split.txt'),   sep=" ", index_col=[0], names=['image_id', 'trainset'])
image_file_names = pd.read_csv(os.path.join(data_dir, 'images.txt'),             sep=" ", index_col=[0], names=['image_id', 'file_name'])
class_labels     = pd.read_csv(os.path.join(data_dir, 'image_class_labels.txt'), sep=" ", index_col=[0], names=['image_id', 'class_id'])
image_attribute_labels = pd.read_csv(os.path.join(data_dir, 'attributes', 'image_attribute_labels.txt'), sep=" ", 
                                     names = ['image_id', 'attribute_id', 'present', 'certainty_id', 'time', 'd1', 'd2'])
image_attribute_labels = image_attribute_labels.merge(train_test_split, on='image_id', how='left' )
image_attribute_labels = image_attribute_labels.merge(class_labels, on='image_id', how='left' )

expert_probabilities = pd.read_csv(os.path.join(data_dir, 'attributes', 'class_attribute_labels_continuous.txt'), sep=" ",   names=[i for i in range(1, 313)]).transpose() / 100

test_attribute_labels  = image_attribute_labels.loc[image_attribute_labels['trainset']==1].copy()
test_attribute_probabilities = test_attribute_labels.pivot_table(index='attribute_id', columns='class_id')['present']



In [3]:
# Class attribute probabilities based on human knowledge

expert_probabilities.transpose()

,1,2,3,4,5,6,7,8,9,10,...,303,304,305,306,307,308,309,310,311,312
0,0.000000,0.029197,0.014599,0.000000,0.598540,0.262774,0.036496,0.058394,0.000000,0.016393,...,0.000000,0.000000,0.212766,0.113475,0.021277,0.319149,0.535714,0.053571,0.214286,0.196429
1,0.044118,0.044118,0.029412,0.014706,0.573529,0.272059,0.000000,0.022059,0.000000,0.000000,...,0.000000,0.022556,0.022556,0.947368,0.000000,0.000000,0.771429,0.038095,0.104762,0.085714
2,0.000000,0.039735,0.033113,0.000000,0.708609,0.145695,0.052980,0.019868,0.000000,0.000000,...,0.022059,0.000000,0.389706,0.029412,0.000000,0.161765,0.710280,0.009346,0.074766,0.205607
3,0.000000,0.014599,0.102190,0.000000,0.000000,0.000000,0.087591,0.255474,0.540146,0.058394,...,0.000000,0.000000,0.885714,0.000000,0.000000,0.021429,0.841270,0.023810,0.031746,0.103175
4,0.000000,0.000000,0.032258,0.000000,0.010753,0.000000,0.000000,0.322581,0.634409,0.000000,...,0.000000,0.000000,0.827273,0.054545,0.000000,0.045455,0.884615,0.000000,0.038462,0.076923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0.027027,0.128378,0.000000,0.000000,0.000000,0.000000,0.763514,0.081081,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.019737,0.000000,0.296053,0.143791,0.228758,0.562092,0.065359
196,0.183007,0.117647,0.000000,0.032680,0.000000,0.000000,0.607843,0.058824,0.000000,0.000000,...,0.000000,0.000000,0.354430,0.044304,0.000000,0.227848,0.167832,0.251748,0.321678,0.258741
197,0.107595,0.367089,0.000000,0.075949,0.006329,0.000000,0.436709,0.006329,0.000000,0.000000,...,0.000000,0.000000,0.143750,0.131250,0.000000,0.418750,0.187919,0.442953,0.187919,0.181208
198,0.000000,0.020690,0.000000,0.020690,0.000000,0.000000,0.882759,0.075862,0.000000,0.000000,...,0.000000,0.013245,0.039735,0.000000,0.000000,0.192053,0.124138,0.344828,0.331034,0.200000


In [21]:
count = 0
for index, present_attribute_ids in expert_probabilities.transpose().iterrows():
    class_id = index + 1
    for test_index, probabilities in expert_probabilities.transpose().iterrows():
        test_class_id = test_index + 1
        probability   = 1 
        for i, attribute_probability in enumerate(present_attribute_ids):
            if attribute_probability != 0:
                probability *= attribute_probability * probabilities[i+1]
            
       
        if probability != 0: 
            print(class_id, test_class_id, probability)
            count += 1
print(count)

2 2 2.5304853416518483e-282
5 5 1.1611854551113887e-306
6 6 7.4468025257733e-273
7 7 5.703494440879241e-296
8 8 3.0569042439255225e-259
10 10 2.7452302523616686e-290
12 12 1.334940725050326e-309
20 20 6.831564403944704e-291
29 29 4.39322487765804e-234
33 33 2.78173226e-315
45 45 3.432178490518353e-288
46 46 3.3559218184254544e-249
52 52 2.4289745752412616e-306
53 53 1.67276e-319
59 59 2.7424136e-317
61 61 1.861804469788136e-303
72 72 2.8553850747236e-310
77 77 5.47845e-319
78 78 2.4332724258707283e-236
81 81 9.007989249624015e-202
84 84 3.695499561354425e-215
85 85 4.37972182329054e-303
93 93 2.1895686266934464e-297
94 94 1.149228632923591e-292
96 96 2.216644686235769e-308
101 101 2.301182490707627e-280
102 102 6.769669202921401e-297
113 113 1.893271217597636e-300
122 122 6.912234135038143e-305
126 126 2.8e-322
127 127 9.53509363286283e-285
139 139 1.002237243e-314
141 141 5.24965e-319
146 146 6.686203594048337e-243
150 150 2.0076798145002713e-268
159 159 3.992986556684411e-300
177 177